In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
image_names = get_png_names("../pooling/data/MEX2/")[0:5]
# image_names = get_png_names("../pooling/data/MEX2/")
# image_indices = random.sample(range(0, len(image_names)), 1000)
# image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../pooling/data/migration_data.json" , image_names)

5 municipalities.


In [4]:
image_names

['../pooling/data/MEX2/484002003/pngs/484002003_2010_1_box484002003_MAY.png',
 '../pooling/data/MEX2/484002004/pngs/484002004_2010_1_box484002004_MAY.png',
 '../pooling/data/MEX2/484004001/pngs/484004001_2010_1_box484004001_MAY.png',
 '../pooling/data/MEX2/484004004/pngs/484004004_2010_1_box484004004_MAY.png',
 '../pooling/data/MEX2/484004006/pngs/484004006_2010_1_box484004006_MAY.png']

In [5]:
# for i in image_names:
#     print(load_inputs(i).shape)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .01)

butler = geozoom_handler(attn_model, 
                         fc_model, device, 
                         criterion, 
                         attn_optimizer, 
                         fc_optimizer, 
                         num_thresholds = 4,
                         reduction_percent = .90,
                         convergence_dims = (358, 284),
                         plot = False, 
                         v = False)

In [16]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [19]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [20]:
for i,o in train_dl:
    print(i)
#     print(load_inputs(i[0]).shape, o)

('../pooling/data/MEX2/484004001/pngs/484004001_2010_1_box484004001_MAY.png',)
('../pooling/data/MEX2/484002003/pngs/484002003_2010_1_box484002003_MAY.png',)
('../pooling/data/MEX2/484002004/pngs/484002004_2010_1_box484002004_MAY.png',)


In [21]:
butler.train_attn_model(train_dl, val_dl)

Epoch:  0
  Training Loss:  13872.653672059378
  Validation Loss:  21.197503089904785
Loss thresholds for training:  [0.0, 5.299375772476196, 10.598751544952393, 15.898127317428589]
Starting from threshold:  3  with value:  15.898127317428589


Epoch:  1
  Training Loss:  13871.62468723456
  Validation Loss:  15.26120376586914
  Moving to threshold:  2   |  Next loss benchmark:  10.598751544952393


Epoch:  2
  Training Loss:  13870.554428080717
  Validation Loss:  3.2871150970458984
  Moving to threshold:  1   |  Next loss benchmark:  5.299375772476196


Epoch:  3
  Training Loss:  13869.754569907984
  Validation Loss:  2.793699264526367
  Moving to threshold:  0   |  Next loss benchmark:  0.0



Image sizes entering FC model: 

{'484002004': [(0, 1794, 0, 2752), (180, 1794, 236, 2712), (0, 1452, 248, 2476), (146, 1452, 31, 2036)], '484002003': [(0, 1488, 0, 3072), (0, 1339, 308, 3072), (0, 1205, 277, 2764), (0, 1084, 249, 2487)], '484004001': [(0, 2732, 0, 2448), (274, 2732, 0, 2203)

AttributeError: 'fc_net' object has no attribute 'sa'

<Figure size 432x288 with 0 Axes>